In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local[1]').setAppName('my app')
sc = SparkContext.getOrCreate(conf=conf)
sc

<SparkContext master=local[1] appName=my app>

In [2]:
raw_content = sc.textFile('stock_prices.csv')

In [3]:
raw_content.count()

1246

In [6]:
raw_content.take(5)


['date,open,high,low,close,volume,ticker',
 '2016-11-02,111.4,112.35,111.23,111.59,28331709,AAPL',
 '2016-11-01,113.46,113.77,110.53,111.49,43825812,AAPL',
 '2016-10-31,113.65,114.23,113.2,113.54,26419398,AAPL',
 '2016-10-28,113.87,115.21,113.45,113.72,37861662,AAPL']

In [8]:
content = raw_content.map(lambda x : x.split(','))
content.take(3)

[['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'],
 ['2016-11-02', '111.4', '112.35', '111.23', '111.59', '28331709', 'AAPL'],
 ['2016-11-01', '113.46', '113.77', '110.53', '111.49', '43825812', 'AAPL']]

In [392]:
header =content.first()
rows_only = content.filter(lambda x: x != header)

In [398]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
import pyspark.sql.functions as F

### a.

In [38]:
df = spark.createDataFrame(rows_only, ['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'])
df.show(3)

+----------+------+------+------+------+--------+------+
|      date|  open|  high|   low| close|  volume|ticker|
+----------+------+------+------+------+--------+------+
|2016-11-02| 111.4|112.35|111.23|111.59|28331709|  AAPL|
|2016-11-01|113.46|113.77|110.53|111.49|43825812|  AAPL|
|2016-10-31|113.65|114.23| 113.2|113.54|26419398|  AAPL|
+----------+------+------+------+------+--------+------+
only showing top 3 rows



In [49]:
df_daily_return = df.withColumn("daily_return", df.close-df.open)
df_daily_return.show(3)

+----------+------+------+------+------+--------+------+--------------------+
|      date|  open|  high|   low| close|  volume|ticker|        daily_return|
+----------+------+------+------+------+--------+------+--------------------+
|2016-11-02| 111.4|112.35|111.23|111.59|28331709|  AAPL| 0.18999999999999773|
|2016-11-01|113.46|113.77|110.53|111.49|43825812|  AAPL| -1.9699999999999989|
|2016-10-31|113.65|114.23| 113.2|113.54|26419398|  AAPL|-0.10999999999999943|
+----------+------+------+------+------+--------+------+--------------------+
only showing top 3 rows



In [289]:
df_daily_return.agg(F.avg(df_daily_return.daily_return)).collect()

[Row(avg(daily_return)=-0.07232931726907578)]

In [397]:
gdf = df_daily_return.groupBy(df_daily_return.ticker)
gdf.agg(F.avg(df_daily_return.daily_return)).sort(F.desc("avg(daily_return)")).collect()

[Row(ticker='BLK', avg(daily_return)=0.15248995983935915),
 Row(ticker='MSFT', avg(daily_return)=0.031164658634538173),
 Row(ticker='AAPL', avg(daily_return)=0.017991967871485902),
 Row(ticker='GOOG', avg(daily_return)=-0.17883534136546128),
 Row(ticker='TSLA', avg(daily_return)=-0.3844578313253008)]

## b.

In [57]:
pos_val = df_daily_return.filter(F.col("daily_return") >=0)
pos_val.show(2)

+----------+------+------+------+------+--------+------+-------------------+
|      date|  open|  high|   low| close|  volume|ticker|       daily_return|
+----------+------+------+------+------+--------+------+-------------------+
|2016-11-02| 111.4|112.35|111.23|111.59|28331709|  AAPL|0.18999999999999773|
|2016-10-26|114.31| 115.7|113.31|115.59|66134219|  AAPL| 1.2800000000000011|
+----------+------+------+------+------+--------+------+-------------------+
only showing top 2 rows



In [433]:
gdf2 = pos_val.groupBy(pos_val.ticker)
gdf2.agg(F.count(pos_val.daily_return)).sort(F.desc("count(daily_return)")).collect()

[Row(ticker='AAPL', count(daily_return)=134),
 Row(ticker='MSFT', count(daily_return)=128),
 Row(ticker='BLK', count(daily_return)=127),
 Row(ticker='GOOG', count(daily_return)=123),
 Row(ticker='TSLA', count(daily_return)=113)]

#### stock with greatest positive daily return = AAPL

## c.

In [76]:
df_price_diff = df.withColumn("price_diff", df.high - df.low)
df_price_diff.select(F.col('ticker'),F.col("price_diff")).show()

+------+------------------+
|ticker|        price_diff|
+------+------------------+
|  AAPL|1.1199999999999903|
|  AAPL| 3.239999999999995|
|  AAPL|1.0300000000000011|
|  AAPL| 1.759999999999991|
|  AAPL|1.7600000000000051|
|  AAPL|2.3900000000000006|
|  AAPL|1.0499999999999972|
|  AAPL|0.7399999999999949|
|  AAPL|0.6299999999999955|
|  AAPL|1.0499999999999972|
|  AAPL| 3.960000000000008|
|  AAPL|0.7599999999999909|
|  AAPL|1.0600000000000023|
|  AAPL|1.0400000000000063|
|  AAPL|1.7199999999999989|
|  AAPL| 1.230000000000004|
|  AAPL| 2.489999999999995|
|  AAPL| 2.030000000000001|
|  AAPL|1.0499999999999972|
|  AAPL| 1.210000000000008|
+------+------------------+
only showing top 20 rows



In [107]:
df_price_diff

DataFrame[date: string, open: string, high: string, low: string, close: string, volume: string, ticker: string, price_diff: double]

In [405]:
df_ts = df_price_diff.withColumn("time_stamp", F.to_timestamp("date"))

In [404]:
a_std = df_ts.groupBy("ticker").agg(stddev("price_diff"))
a_std.sort(F.desc("stddev_samp(price_diff)")).show()

+------+-----------------------+
|ticker|stddev_samp(price_diff)|
+------+-----------------------+
|  GOOG|      6.407324009690074|
|  TSLA|     3.3510576333377227|
|   BLK|      3.055361869614563|
|  AAPL|     0.8364876885309602|
|  MSFT|     0.4053488922473723|
+------+-----------------------+



### most volatile stock = GOOG

## d.

In [183]:
df_daily_return.show()

+----------+------+------+------+------+--------+------+--------------------+
|      date|  open|  high|   low| close|  volume|ticker|        daily_return|
+----------+------+------+------+------+--------+------+--------------------+
|2016-11-02| 111.4|112.35|111.23|111.59|28331709|  AAPL| 0.18999999999999773|
|2016-11-01|113.46|113.77|110.53|111.49|43825812|  AAPL| -1.9699999999999989|
|2016-10-31|113.65|114.23| 113.2|113.54|26419398|  AAPL|-0.10999999999999943|
|2016-10-28|113.87|115.21|113.45|113.72|37861662|  AAPL|-0.15000000000000568|
|2016-10-27|115.39|115.86| 114.1|114.48|34562045|  AAPL| -0.9099999999999966|
|2016-10-26|114.31| 115.7|113.31|115.59|66134219|  AAPL|  1.2800000000000011|
|2016-10-25|117.95|118.36|117.31|118.25|48128970|  AAPL| 0.29999999999999716|
|2016-10-24| 117.1|117.74|   117|117.65|23538673|  AAPL|  0.5500000000000114|
|2016-10-21|116.81|116.91|116.28| 116.6|23192665|  AAPL|-0.21000000000000796|
|2016-10-20|116.86|117.38|116.33|117.06|24125801|  AAPL| 0.20000

In [184]:
df_daily_return.count()

1245

In [328]:
df_daily_return = df_daily_return.withColumn("time_stamp", F.to_timestamp("date"))

In [374]:
days = df_daily_return.filter(F.col("ticker") == "AAPL")
days.count()

249

In [376]:
AAPL_pos = df_daily_return.filter((F.col("ticker") == "AAPL") & (F.col("daily_return") >=0))
GOOG_pos = df_daily_return.filter((F.col("ticker") == "GOOG") & (F.col("daily_return") <0))

In [378]:
AAPL_pos.count()

134

In [377]:
GOOG_pos.count()

126

In [335]:
AAPL_pos.show(2)

+----------+------+------+------+------+--------+------+-------------------+-------------------+
|      date|  open|  high|   low| close|  volume|ticker|       daily_return|         time_stamp|
+----------+------+------+------+------+--------+------+-------------------+-------------------+
|2016-11-02| 111.4|112.35|111.23|111.59|28331709|  AAPL|0.18999999999999773|2016-11-02 00:00:00|
|2016-10-26|114.31| 115.7|113.31|115.59|66134219|  AAPL| 1.2800000000000011|2016-10-26 00:00:00|
+----------+------+------+------+------+--------+------+-------------------+-------------------+
only showing top 2 rows



In [406]:
A_G = AAPL_pos.join(GOOG_pos, AAPL_pos.time_stamp == GOOG_pos.time_stamp, "inner" )

In [407]:
A_G.count()

51

In [408]:
percentage = str((A_G.count()/days.count())*100) + "%"
percentage

'20.481927710843372%'

### percentage = 20.481927710843372%

## e.

In [198]:
df_ts.first()

Row(date='2016-11-02', open='111.4', high='112.35', low='111.23', close='111.59', volume='28331709', ticker='AAPL', price_diff=1.1199999999999903, time_stamp=datetime.datetime(2016, 11, 2, 0, 0))

In [409]:
df_ts = df_ts.withColumn("high", df_ts["high"].cast('float'))
df_ts = df_ts.withColumn("low", df_ts["low"].cast("float"))
df_ts = df_ts.withColumn("close", df_ts["close"].cast("float"))

In [410]:
_max = df_ts.groupBy(F.col("ticker").alias("Name")).agg(F.max("close").alias("max_close"))
_min = df_ts.groupBy(F.col("ticker").alias("Name")).agg(F.min("close").alias("min_close"))

In [411]:
_max.show()

+----+---------+
|Name|max_close|
+----+---------+
| BLK|    376.0|
|AAPL|   120.57|
|TSLA|   265.42|
|GOOG|   813.11|
|MSFT|     61.0|
+----+---------+



In [417]:
_max.collect()

[Row(Name='BLK', max_close=376.0),
 Row(Name='AAPL', max_close=120.56999969482422),
 Row(Name='TSLA', max_close=265.4200134277344),
 Row(Name='GOOG', max_close=813.1099853515625),
 Row(Name='MSFT', max_close=61.0)]

In [418]:
a = df_ts.filter((F.col("ticker") == "BLK") & (F.col("close") == 376)).select('date','ticker', 'close')
b = df_ts.filter((F.col("ticker") == "AAPL") & (F.col("close") == 120.56999969482422)).select('date','ticker', 'close')
c = df_ts.filter((F.col("ticker") == "TSLA") & (F.col("close") == 265.4200134277344)).select('date','ticker', 'close')
d = df_ts.filter((F.col("ticker") == "GOOG") & (F.col("close") == 813.1099853515625)).select('date','ticker', 'close')
e = df_ts.filter((F.col("ticker") == "MSFT") & (F.col("close") == 61.0)).select('date','ticker', 'close')

## highest closing price and their dates

In [420]:
a.union(b).union(c).union(d).union(e).collect()

[Row(date='2016-08-30', ticker='BLK', close=376.0),
 Row(date='2015-11-09', ticker='AAPL', close=120.56999969482422),
 Row(date='2016-04-06', ticker='TSLA', close=265.4200134277344),
 Row(date='2016-10-24', ticker='GOOG', close=813.1099853515625),
 Row(date='2016-10-24', ticker='MSFT', close=61.0)]

In [421]:
_min.collect()

[Row(Name='BLK', min_close=289.7200012207031),
 Row(Name='AAPL', min_close=90.33999633789062),
 Row(Name='TSLA', min_close=143.6699981689453),
 Row(Name='GOOG', min_close=668.260009765625),
 Row(Name='MSFT', min_close=48.43000030517578)]

In [424]:
a = df_ts.filter((F.col("ticker") == "BLK") & (F.col("close") == 289.7200012207031)).select('date','ticker', 'close')
b = df_ts.filter((F.col("ticker") == "AAPL") & (F.col("close") == 90.33999633789062)).select('date','ticker', 'close')
c = df_ts.filter((F.col("ticker") == "TSLA") & (F.col("close") == 143.6699981689453)).select('date','ticker', 'close')
d = df_ts.filter((F.col("ticker") == "GOOG") & (F.col("close") == 668.260009765625)).select('date','ticker', 'close')
e = df_ts.filter((F.col("ticker") == "MSFT") & (F.col("close") == 48.43000030517578)).select('date','ticker', 'close')

### Lowest closing price and their dates

In [425]:
a.union(b).union(c).union(d).union(e).collect()

[Row(date='2016-01-25', ticker='BLK', close=289.7200012207031),
 Row(date='2016-05-12', ticker='AAPL', close=90.33999633789062),
 Row(date='2016-02-10', ticker='TSLA', close=143.6699981689453),
 Row(date='2016-06-27', ticker='GOOG', close=668.260009765625),
 Row(date='2016-06-27', ticker='MSFT', close=48.43000030517578)]

## f.

In [428]:
df_ts = df_ts.withColumn("close", df_ts["close"].cast("float"))
df_ts = df_ts.withColumn("volume", df_ts["volume"].cast("float"))

In [429]:
df_ts = df_ts.withColumn("freq", df_ts["close"]* df_ts["volume"])

In [431]:
df_ts.groupBy("ticker").agg(F.avg("freq")).sort(F.desc("avg(freq)")).collect()

[Row(ticker='AAPL', avg(freq)=4112931005.686747),
 Row(ticker='MSFT', avg(freq)=1725271740.5301204),
 Row(ticker='GOOG', avg(freq)=1349722722.698795),
 Row(ticker='TSLA', avg(freq)=928122117.1405623),
 Row(ticker='BLK', avg(freq)=233390101.17269075)]

##### most traded stock = AAPL

## g.

In [385]:
df_month = df_ts.groupBy(F.month("time_stamp").alias("month")).agg(F.avg("freq").alias("avg_freq")).orderBy("avg_freq").sort(F.desc("avg_freq"))
df_month.collect()

[Row(month=1, avg_freq=2394495128.0842104),
 Row(month=2, avg_freq=1903065293.6),
 Row(month=4, avg_freq=1851668877.0285714),
 Row(month=12, avg_freq=1808507178.9818182),
 Row(month=9, avg_freq=1693282200.8380952),
 Row(month=11, avg_freq=1637244437.6470587),
 Row(month=6, avg_freq=1587477849.3090909),
 Row(month=10, avg_freq=1586737538.4380953),
 Row(month=5, avg_freq=1550525001.6761904),
 Row(month=3, avg_freq=1529972108.6545455),
 Row(month=7, avg_freq=1453321110.48),
 Row(month=8, avg_freq=1158615777.3217392)]

### month with highest trading frequency on average = January

### h.

In [389]:
df_ts.withColumn("week_strt_day",F.date_sub(F.next_day(F.col("time_stamp"),"sunday"),7)).groupBy("week_strt_day").agg(F.avg("freq").alias("avg_freq")).sort(F.desc("avg_freq")).collect()

[Row(week_strt_day=datetime.date(2015, 12, 13), avg_freq=2506002214.4),
 Row(week_strt_day=datetime.date(2016, 9, 11), avg_freq=2501410097.28),
 Row(week_strt_day=datetime.date(2016, 1, 3), avg_freq=2498076842.24),
 Row(week_strt_day=datetime.date(2016, 1, 31), avg_freq=2469173722.24),
 Row(week_strt_day=datetime.date(2016, 1, 24), avg_freq=2451333396.48),
 Row(week_strt_day=datetime.date(2016, 1, 10), avg_freq=2356512701.44),
 Row(week_strt_day=datetime.date(2016, 4, 24), avg_freq=2286831214.72),
 Row(week_strt_day=datetime.date(2016, 1, 17), avg_freq=2241448183.2),
 Row(week_strt_day=datetime.date(2016, 10, 23), avg_freq=2082543161.6),
 Row(week_strt_day=datetime.date(2016, 2, 7), avg_freq=2081954122.88),
 Row(week_strt_day=datetime.date(2016, 6, 19), avg_freq=2061030392.96),
 Row(week_strt_day=datetime.date(2016, 4, 17), avg_freq=2020532625.28),
 Row(week_strt_day=datetime.date(2015, 11, 29), avg_freq=2006943319.68),
 Row(week_strt_day=datetime.date(2016, 7, 24), avg_freq=1902865854

In [390]:
import datetime
datetime.date(2015, 12, 13).isocalendar()[1]


50

### week 50